# Updates

In [1]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [19]:
from casestudy.update.baseframe import make
from casestudy.update.helpers import test_data_is_timely

In [20]:
# def test_data_is_timely(baseframe):    
#     today = dt.now()
#     expired = [region_id for region_id, df_group in baseframe.groupby('region_id') if (today - df_group.date.max()).days > 3]
#     print (expired)
#     try:
#         assert len(expired) == 0
#     except Exception as e:
#         raise Exception(' ...Failing Regions: {}'.format(' '.join(expired))) from e

In [11]:
baseframe = make()

initializing
adding fatalities
adding cases
adding tests
adding pollutants
adding measurements
backfill time-static data
adding strindex
adding Google Mobility
adding Apple Mobility
adding Causes of Death
adding Travel popularity
adding GDP
COMPLETE


In [21]:
test_data_is_timely(baseframe)

['497']
497


Exception:  ...Failing Regions: 497